In [1]:
def filter_sto_by_time(input_file, output_file, start_time=35.003, end_time=50.005):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        header_complete = False
        for line in infile:
            if not header_complete:
                # Write header lines unchanged
                outfile.write(line)
                if "endheader" in line:
                    header_complete = True
            else:
                # Process the data lines
                values = line.strip().split("\t")
                try:
                    # Assuming the first column is time, convert it to float
                    time_value = float(values[0])
                    # Write the line only if the time is within the specified range
                    if start_time <= time_value <= end_time:
                        outfile.write(line)
                except ValueError:
                    pass  # Skip lines that do not have a valid time value

# 使用範例
input_file = 'data/emg_data_with_labels.sto'  # 你的 .sto 文件路徑
output_file = 'output/emg_filtered.sto'  # 輸出的 .sto 文件路徑
filter_sto_by_time(input_file, output_file)